In [1]:
import pandas as pd
from utils.Points import PlayerPoints, PlayerServes
from utils.Tools import ResumePointsStats,ResumeServeStats, ReturnTerrain
import csv

In [2]:
pd.options.display.max_columns = None

# Tratamento

In [3]:
matches = pd.read_csv('charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)

In [4]:
matches['Surface'].unique()

array(['Hard', 'Clay', 'Grass', 'Carpet', 'Clay ', nan, 'Hard Indoor',
       'Hard ', 'Indoor', 'Indoor Hard', 'Indoor Clay', 'Hard (indoor)',
       'Hard Court', 'Hard (outdoor)'], dtype=object)

In [5]:
H = ['Hard', 'Hard Indoor',
       'Hard ', 'Indoor Hard', 'Hard (indoor)',
       'Hard Court', 'Hard (outdoor)']

C = ['Clay', 'Clay ','Indoor Clay']

In [6]:
# Tratando erros/ agrupando as superficies
matches = matches.dropna(subset=['Surface'])

matches.loc[(matches['Surface'].isin(C)), ['Surface']] = 'Clay'
matches.loc[(matches['Surface'].isin(H)), ['Surface']] = 'Hard'
matches = matches[matches['Surface'] != 'Carpet']

matches['Surface'].unique()

array(['Hard', 'Clay', 'Grass', 'Indoor'], dtype=object)

In [7]:
# Filtrando os jogos de cada jogador

Federer_Matches = matches[(matches['Player 1'] == 'Roger Federer') | (matches['Player 2'] == 'Roger Federer')]
Nadal_Matches = matches[(matches['Player 1'] == 'Rafael Nadal') | (matches['Player 2'] == 'Rafael Nadal')]
Djokovic_Matches = matches[(matches['Player 1'] == 'Novak Djokovic') | (matches['Player 2'] == 'Novak Djokovic')]

In [8]:
# Pegando todos os Ids dos jogos de cada jogador

Nadal_Id = Nadal_Matches['match_id'].unique()
Djokovic_Id = Djokovic_Matches['match_id'].unique()
Federer_Id = Federer_Matches['match_id'].unique()

In [9]:
FedererAux = Federer_Matches[['match_id','Player 1', 'Player 2','Surface']]
NadalAux = Nadal_Matches[['match_id','Player 1', 'Player 2','Surface']]
DjokovicAux = Djokovic_Matches[['match_id','Player 1', 'Player 2','Surface']]

In [10]:
#Carregando datasets de pontos

points2010 = pd.read_csv('charting-m-points-2010s.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
points2020 = pd.read_csv('charting-m-points-2020s.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
points_to_2009 = pd.read_csv('charting-m-points-to-2009.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)

C:\Users\cimin\AppData\Local\Temp\ipykernel_26316\1472645856.py:3: DtypeWarning: Columns (8,9,29) have mixed types. Specify dtype option on import or set low_memory=False.
  points2010 = pd.read_csv('charting-m-points-2010s.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
C:\Users\cimin\AppData\Local\Temp\ipykernel_26316\1472645856.py:4: DtypeWarning: Columns (8,9,29) have mixed types. Specify dtype option on import or set low_memory=False.
  points2020 = pd.read_csv('charting-m-points-2020s.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
C:\Users\cimin\AppData\Local\Temp\ipykernel_26316\1472645856.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  points_to_2009 = pd.read_csv('charting-m-points-to-2009.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)


In [11]:
#Juntando todos os pontos em um só dataset

Points = pd.concat([points_to_2009,points2010,points2020],axis=0).reset_index(drop=True)

In [12]:
#Para cada jogador, retorna todos os pontos disputados em cada um de seus jogos

Federer_Games_Points = pd.merge(Points,FedererAux, on='match_id')
Nadal_Games_Points = pd.merge(Points,NadalAux, on='match_id')
Djokovic_Games_Points = pd.merge(Points,DjokovicAux, on='match_id')

In [13]:
Federer_Games_Points

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,TBpt,Svr,Ret,Serving,1st,2nd,Notes,1stSV,2ndSV,1stIn,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,Player 1,Player 2,Surface
0,20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...,1,0,0,0.0,0.0,0-0,1 (1),1,0,NaN,1,2,RF,6d,5b18f2b3b2b2f3s1f1*,NaN,0,0.0,0,1.0,False,False,True,False,False,False,1,1,9.0,Roger Federer,Nikolay Davydenko,Hard
1,20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...,2,0,0,0.0,0.0,15-0,1 (2),1,0,NaN,1,2,RF,6*,NaN,NaN,0,NaN,1,NaN,True,False,False,False,False,False,1,1,1.0,Roger Federer,Nikolay Davydenko,Hard
2,20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...,3,0,0,0.0,0.0,30-0,1 (3),1,0,NaN,1,2,RF,4r38f3b1d#,NaN,NaN,0,NaN,1,NaN,False,False,False,True,False,False,1,1,3.0,Roger Federer,Nikolay Davydenko,Hard
3,20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...,4,0,0,0.0,0.0,40-0,1 (4),1,0,NaN,1,2,RF,4*,NaN,NaN,0,NaN,1,NaN,True,False,False,False,False,False,1,1,1.0,Roger Federer,Nikolay Davydenko,Hard
4,20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...,5,0,0,1.0,0.0,0-0,2 (1),1,0,NaN,2,1,ND,6s38b2f2f2b1*,NaN,NaN,0,NaN,1,NaN,False,False,True,False,False,False,1,0,6.0,Roger Federer,Nikolay Davydenko,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99478,20200120-M-Australian_Open-R128-Roger_Federer-...,135,2,0,5.0,2.0,0-0,25 (1),1.0,0,NaN,1,2,RF,6n,6b3d#,NaN,0,0.0,0,1.0,False,False,False,True,False,False,1,1,1,Roger Federer,Steve Johnson,Hard
99479,20200120-M-Australian_Open-R128-Roger_Federer-...,136,2,0,5.0,2.0,15-0,25 (2),1.0,0,NaN,1,2,RF,4n,6f29f38b28b1d@,NaN,0,0.0,0,1.0,False,False,False,False,True,False,2,0,4,Roger Federer,Steve Johnson,Hard
99480,20200120-M-Australian_Open-R128-Roger_Federer-...,137,2,0,5.0,2.0,15-15,25 (3),1.0,0,NaN,1,2,RF,6m2o=38m2o38*,NaN,NaN,0,NaN,1,NaN,False,False,True,False,False,False,1,1,5,Roger Federer,Steve Johnson,Hard
99481,20200120-M-Australian_Open-R128-Roger_Federer-...,138,2,0,5.0,2.0,30-15,25 (4),1.0,0,NaN,1,2,RF,4b3d#,NaN,NaN,0,NaN,1,NaN,False,False,False,True,False,False,1,1,1,Roger Federer,Steve Johnson,Hard


In [14]:
# Divide os pontos  em vencidos/perdidos por cada jogador

Federer_Points_Won, Federer_Points_Lost = PlayerPoints('Roger Federer', Federer_Games_Points, Federer_Id)
Nadal_Points_Won, Nadal_Points_Lost = PlayerPoints('Rafael Nadal', Nadal_Games_Points, Nadal_Id)
Djokovic_Points_Won, Djokovic_Points_Lost = PlayerPoints('Novak Djokovic', Djokovic_Games_Points, Djokovic_Id)

In [15]:
# Divide os pontos em saque/receives para cada jogador

Federer_Serves, Federer_Receives = PlayerServes('Roger Federer', Federer_Games_Points, Federer_Id)
Nadal_Serves, Nadal_Receives = PlayerServes('Rafael Nadal', Nadal_Games_Points, Nadal_Id)
Djokovic_Serves, Djokovic_Receives = PlayerServes('Novak Djokovic', Djokovic_Games_Points, Djokovic_Id)

In [16]:
# Retorna os pontos vencidos nos quais o jogadore estavam no saque

Federer_Serve_Points = Federer_Serves[Federer_Serves['Svr'] == Federer_Serves['PtWinner']]
Nadal_Serve_Points = Nadal_Serves[Nadal_Serves['Svr'] == Nadal_Serves['PtWinner']]
Djokovic_Serve_Points = Djokovic_Serves[Djokovic_Serves['Svr'] == Djokovic_Serves['PtWinner']]

In [17]:
# Retorna os pontos vencidos nos quais o jogador estava recebendo
Federer_Receive_Points = Federer_Receives[Federer_Receives['Ret'] == Federer_Receives['PtWinner']]
Nadal_Receive_Points = Nadal_Receives[Nadal_Receives['Ret'] == Nadal_Receives['PtWinner']]
Djokovic_Receive_Points = Djokovic_Receives[Djokovic_Receives['Ret'] == Djokovic_Receives['PtWinner']]

In [18]:
Federer_Serves['isAce'].value_counts()

isAce
False    43343
True      4815
Name: count, dtype: int64

In [19]:
Federer_Points_Won[Federer_Points_Won['isDouble'] == True]

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,TBpt,Svr,Ret,Serving,1st,2nd,Notes,1stSV,2ndSV,1stIn,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,Player 1,Player 2,Surface
29,20210705-M-Wimbledon-R16-Lorenzo_Sonego-Roger_...,64,0,0,5.0,5.0,40-0,11 (4),1,0,NaN,1,2,LS,4n,4w,NaN,0,0.0,0,0.0,False,False,False,False,False,True,2,0,0,Lorenzo Sonego,Roger Federer,Grass
39,20210705-M-Wimbledon-R16-Lorenzo_Sonego-Roger_...,80,0,0,5.0,5.0,40-AD,11 (20),1,0,NaN,1,2,LS,4n,4n,Roof is closed,0,0.0,0,0.0,False,False,False,False,False,True,2,0,0,Lorenzo Sonego,Roger Federer,Grass
103,20210703-M-Wimbledon-R32-Cameron_Norrie-Roger_...,2,0,0,0.0,0.0,15-0,1 (2),1,0,NaN,1,2,CN,c4d,5d,Let on 4.,0,0.0,0,0.0,False,False,False,False,False,True,2,0,0,Cameron Norrie,Roger Federer,Grass
104,20210703-M-Wimbledon-R32-Cameron_Norrie-Roger_...,3,0,0,0.0,0.0,15-15,1 (3),1,0,NaN,1,2,CN,4x,5d,NaN,0,0.0,0,0.0,False,False,False,False,False,True,2,0,0,Cameron Norrie,Roger Federer,Grass
105,20210703-M-Wimbledon-R32-Cameron_Norrie-Roger_...,5,0,0,0.0,0.0,30-30,1 (5),1,0,NaN,1,2,CN,6w,4d,NaN,0,0.0,0,0.0,False,False,False,False,False,True,2,0,0,Cameron Norrie,Roger Federer,Grass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52759,19990716-M-Davis_Cup_World_Group_QF-RR-Roger_F...,59,0,0,2.0,3.0,30-30,6 (5),1,0,NaN,2,1,XM,6d,6n,NaN,0,0.0,0,0.0,False,False,False,False,False,True,1,0,0.0,Roger Federer,Xavier Malisse,Clay
52782,19990716-M-Davis_Cup_World_Group_QF-RR-Roger_F...,102,0,0,5.0,4.0,40-40,10 (19),1,0,NaN,2,1,XM,6d,4n,NaN,0,0.0,0,0.0,False,False,False,False,False,True,1,0,0.0,Roger Federer,Xavier Malisse,Clay
52785,19990716-M-Davis_Cup_World_Group_QF-RR-Roger_F...,109,1,0,0.0,1.0,0-0,12 (1),1,0,NaN,2,1,XM,4w,6n,NaN,0,0.0,0,0.0,False,False,False,False,False,True,1,0,0.0,Roger Federer,Xavier Malisse,Clay
52829,19990716-M-Davis_Cup_World_Group_QF-RR-Roger_F...,204,1,1,4.0,4.0,40-0,28 (4),1,0,NaN,2,1,XM,6n,4n,NaN,0,0.0,0,0.0,False,False,False,False,False,True,1,0,0.0,Roger Federer,Xavier Malisse,Clay


In [22]:
Hard, Clay, Grass = ReturnTerrain(Federer_Games_Points)

(                                                match_id   Pt  Set1  Set2  \
 0      20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...    1     0     0   
 1      20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...    2     0     0   
 2      20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...    3     0     0   
 3      20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...    4     0     0   
 4      20091128-M-Tour_Finals-SF-Roger_Federer-Nikola...    5     0     0   
 ...                                                  ...  ...   ...   ...   
 99478  20200120-M-Australian_Open-R128-Roger_Federer-...  135     2     0   
 99479  20200120-M-Australian_Open-R128-Roger_Federer-...  136     2     0   
 99480  20200120-M-Australian_Open-R128-Roger_Federer-...  137     2     0   
 99481  20200120-M-Australian_Open-R128-Roger_Federer-...  138     2     0   
 99482  20200120-M-Australian_Open-R128-Roger_Federer-...  139     2     0   
 
        Gm1  Gm2    Pts     Gm# TbSet TB?  TBpt  Svr  Ret Serv

# Visualização

In [20]:
GrassMatches = len(matches[matches['Surface'] == 'Grass'])
ClayMatches = len(matches[matches['Surface'] == 'Clay'])
HardMatches = len(matches[matches['Surface'] == 'Hard'])

In [21]:
pd.options.display.max_columns = None